In [10]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [13]:
import pandas as pd
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
print('Размерность датасета с рейтингами:', ratings_df.shape)

Размерность датасета с рейтингами: (100836, 4)


In [5]:
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
n_ratings = len(ratings_df)
print("Число уникальных юзеров:", n_users)
print("Число уникальных фильмов:", n_items)
print("Матрица предпочтений будет иметь", n_users*n_items, 'элементов.')
print("Количество оценок:", len(ratings_df))
print("Заполнено", len(ratings_df) / (n_users*n_items) * 100, '% от матрицы предпочтений')

Число уникальных юзеров: 610
Число уникальных фильмов: 9724
Матрица предпочтений будет иметь 5931640 элементов.
Количество оценок: 100836
Заполнено 1.6999683055613624 % от матрицы предпочтений


In [6]:
user_freq = ratings_df[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
user_freq.head()

,userId,n_ratings
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [7]:
movie_stats = ratings_df.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()
movie_stats

,count,mean
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429
...,...,...
193581,1,4.000000
193583,1,3.500000
193585,1,3.500000


In [ ]:
'''
СОздаем матрицу пользовательских элементов
'''

In [11]:
def create_matrix(df):  
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    # Сопоставляем идентификаторы с индексами
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
      
    # Сопоставляем индексы с идентификаторами))
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
      
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
      
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
  

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings_df)
  

In [12]:
# поиск похожих фильмов с использованием KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False): 
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [18]:
movie_titles = dict(zip(movies_df['movieId'], movies_df['title']))
  
movie_id = 228
  
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
  
print(f"Если понравился {movie_title}, то попробуй:")
for i in similar_ids:
    print(movie_titles[i])

Если понравился Destiny Turns on the Radio (1995), то попробуй:
Stuart Saves His Family (1995)
It's My Party (1996)
Boys of St. Vincent, The (1992)
Nobody's Fool (1994)
Bad Girls (1994)
Kiss of Death (1995)
NeverEnding Story III, The (1994)
Unstrung Heroes (1995)
Amazing Panda Adventure, The (1995)
Man Without a Face, The (1993)
